Euclid GC Fisher Matrix Code

Author: Alkistis Pourtsidou, ICG Portsmouth

Using part of http://camb.readthedocs.io/en/latest/CAMBdemo.html 

To run this Jupyter notebook you need to have CAMB and the CAMB python package 
installed. In order to install the CAMB python package on your computer follow
the instructions in http://camb.readthedocs.io/en/latest/

In [1]:
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

print('Using CAMB installed at '+ os.path.realpath(os.path.join(os.getcwd(),'..')))
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower

Using CAMB installed at /Users/alkistis/workspace


In [2]:
import scipy
from scipy.interpolate import interp1d
from __future__ import division

In [3]:
from scipy import integrate
from scipy import linalg

pi=np.pi

In [4]:
#Fiducial cosmological parameters

hubble=0.67
omegab=0.022445/hubble**2
omegac=0.121203/hubble**2
om0=omegac+omegab
H00=100*hubble
Ass=2.1265e-9
nss=0.96

kpiv=0.05

gamma=0.545

c=3e5

print om0

0.32


In [5]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [6]:
#calculate results for these parameters
results = camb.get_results(pars)
#print pars

In [7]:
#Get matter power spectrum at z=0: P(k,z=0)

#Not non-linear corrections couples to smaller scales than you want
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Just linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=1.0, npoints = 200)

In [8]:
#Construct P(k,z=0) interpolating function
Pkz0 = interp1d(kh, pk[0])

In [9]:
#get fiducial σ8 (at z=0)
sig8 = results.get_sigma8()[0]
print sig8

0.830287480109


In [10]:
#Redshift bins
zlist = np.arange(0.7,2.1,0.1)
ztest = zlist[0]
Nzbins = len(zlist)

biaslist = [1.083, 1.125, 1.104, 1.126, 1.208, 1.243, 1.282, 1.292, 1.363, 1.497, 1.486, \
            1.491, 1.573, 1.568]

kmin = 0.001
kmax = 0.2

dn3 = [2434.28, 4364.812, 4728.559, 4825.798, 4728.797, 4507.625, 4269.851, 3720.657, 3104.309, \
       2308.975, 1541.831, 1474.707, 893.716, 497.613]

print zlist
print len(zlist), len(dn3)

[ 0.7  0.8  0.9  1.   1.1  1.2  1.3  1.4  1.5  1.6  1.7  1.8  1.9  2. ]
14 14


In [11]:
#Spatially flat Universe

#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))
def Hz(zc):
    return Ez(zc)*H00

#Define the cosmological distances
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)
def DA(zc):
    return rcom(zc)/(1+zc)

print rcom(ztest)
print DA(ztest)

2596.15131772
1527.14783395


In [12]:
#LCDM growth rate
def fg(zz):
    omz=om0*pow(1+zz,3)/pow(Ez(zz),2)
    return pow(omz,gamma)

print fg(ztest)

0.822099034946


In [13]:
#Get the growth factor 
def Dg_dz(zz):
    return -fg(zz)/(1+zz)

def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(ans)

print Dgz(ztest), Dgz(0)

0.693913449649 1.0


In [14]:
#def sig8z(zc):
#    return Dgz(zc)*sig8

#print sig8z(ztest)

In [15]:
#Construct P_gg(k,μ,z)
def Pgg(kk,mu,zc):
    return pow(bg+fg(zc)*mu**2,2)*pow(Dgz(zc),2)*Pkz0(kk)

In [16]:
#Euclid
Area=15000.0 #deg^2
omegatot = Area*pow(pi/180,2)

Dzbin = 0.1

In [17]:
def photoz(kk,mu,zc):
    sigz = 0.001*(1+zc)
    return np.exp(-pow(kk*mu,2)*pow(c*sigz,2)/pow(Hz(zc),2))

print photoz(0.5,0.5,ztest)

0.200298557962


In [18]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

def Pshot(zc):
    return 1/nbar

#effective volume; the hubble**3 factors are for units consistency
def Veff(kk,mu,zc):
    return hubble**3*Vsur(zc)*(Pgg(kk,mu,zc)*photoz(kk,mu,zc)/(Pgg(kk,mu,zc)*photoz(kk,mu,zc)\
                                                     +hubble**3*Pshot(zc)))**2

print ztest, "%.4g" %  Vsur(ztest)

0.7 9.184e+09


In [19]:
#Fisher matrix parameters 
params = ["0:lnH","1:lnDA","2:lnfsig8","3:lnbsig8","4:Ps"]

In [20]:
#Fisher matrix derivatives

def dlnP_dlnfsig8(kk,mu,zc):
    return 2*mu**2*fg(zc)/(bg+mu**2*fg(zc))

def dlnP_dlnbsig8(kk,mu,zc):
    return 2*bg/(bg+mu**2*fg(zc))

def dlnP_dlnDA(kk,mu,zc):
    dk = (kmax-kmin)/400
    return -2.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            -kk*(1-mu**2)*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dlnH(kk,mu,zc):
    dk = (kmax-kmin)/400
    return 1.0+4*mu**2*(1-mu**2)*fg(zc)/(bg+mu**2*fg(zc))\
            +kk*mu**2*(1/Pkz0(kk))*(Pkz0(kk+dk)-Pkz0(kk-dk))/(2*dk)

def dlnP_dPs(kk,mu,zc):
    return 1.0/(Pgg(kk,mu,zc)*photoz(kk,mu,zc))

In [21]:
def dF(kk,mu):
    return (1./(8*pi*pi))*pow(kk,2)*deriv_i(kk,mu,zc)*deriv_j(kk,mu,zc)*Veff(kk,mu,zc)    

In [22]:
#2D integration function
def integrate2D(dfun, kgrid, mugrid):
    
    muint = [scipy.integrate.simps(dfun.T[i], mugrid) for i in range(kgrid.size)]
    return scipy.integrate.simps(muint, kgrid)

In [23]:
mugrid = np.linspace(-1., 1., 200) 

In [85]:
fo = open("errors.txt", "w")

#%%time
#   Fisher matrix   # 

Npar = len(params)
#create array of zeros
s = (Npar,Npar)

for zi in range(0,Nzbins):
    
    zc = zlist[zi]
    bg = biaslist[zi]
    zmin = zc-Dzbin/2
    zmax = zc+Dzbin/2
     
    dVdz = (4*pi/3)*((1+zmax)**3*DA(zmax)**3-(1+zmin)**3*DA(zmin)**3)/(4*pi*pow(180/pi,2))
    nbar = Dzbin*dn3[zi]/dVdz
    
    kgrid = np.linspace(kmin, kmax, 400)
    K, MU = np.meshgrid(kgrid, mugrid)
    Fishermat = np.zeros(s)
    for i in range(0,Npar):
        def deriv_i(kk,mu,zc):
            if i==0:  return dlnP_dlnH(kk,mu,zc)
            elif i==1:  return dlnP_dlnDA(kk,mu,zc)
            elif i==2:  return dlnP_dlnfsig8(kk,mu,zc)
            elif i==3:  return dlnP_dlnbsig8(kk,mu,zc)
            elif i==4:  return dlnP_dPs(kk,mu,zc)
            else: print "Error: index out of range"
        for  j in range(0,Npar):
            if j>=i:
                def deriv_j(kk,mu,zc):
                    if j==0:  return dlnP_dlnH(kk,mu,zc)
                    elif j==1:  return dlnP_dlnDA(kk,mu,zc)
                    elif j==2:  return dlnP_dlnfsig8(kk,mu,zc)
                    elif j==3:  return dlnP_dlnbsig8(kk,mu,zc)
                    elif j==4:  return dlnP_dPs(kk,mu,zc)
                    else: print "Error: index out of range" 
                Fishermat[i][j] = integrate2D(dF(K,MU),kgrid,mugrid)                
            else: Fishermat[i,j] = Fishermat[j,i]
            #if zi==0: print i,j,Fishermat[i][j]
            
    zbin = str(zc)
    lnDerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[1,1]))
    lnHerr = str("%e" % np.sqrt(linalg.inv(Fishermat)[0,0]))
    lnfsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[2,2]))
    lnbsig8err = str("%e" % np.sqrt(linalg.inv(Fishermat)[3,3]))
    Pserr = str("%e" % np.sqrt(linalg.inv(Fishermat)[4,4]))   
    
    print zbin, lnDerr, lnHerr, lnfsig8err, lnbsig8err, Pserr

    
    outp = zbin + "\t" + lnDerr + "\t" + lnHerr + "\t" + lnfsig8err + "\t" + lnbsig8err + \
    "\t" + Pserr + "\n"
    fo.write(outp)
    
fo.close()    

0.7 7.944782e-03 9.997590e-03 1.496124e-02 8.581684e-03 1.510869e+01
0.8 6.792028e-03 8.647442e-03 1.292743e-02 7.332228e-03 1.258348e+01
0.9 6.546903e-03 8.234876e-03 1.194821e-02 7.117024e-03 1.092848e+01
1.0 6.399109e-03 8.020966e-03 1.169220e-02 6.962693e-03 1.022737e+01
1.1 6.278073e-03 7.939504e-03 1.212992e-02 6.784590e-03 1.039272e+01
1.2 6.295908e-03 7.948901e-03 1.235481e-02 6.796219e-03 1.016495e+01
1.3 6.355553e-03 8.017418e-03 1.272213e-02 6.848282e-03 1.006584e+01
1.4 6.662757e-03 8.322162e-03 1.329261e-02 7.184250e-03 1.003468e+01
1.5 7.023551e-03 8.781900e-03 1.466251e-02 7.526674e-03 1.081104e+01
1.6 7.635647e-03 9.629530e-03 1.741428e-02 8.084216e-03 1.283787e+01
1.7 9.412093e-03 1.152880e-02 2.090488e-02 9.957969e-03 1.499829e+01
1.8 9.910497e-03 1.203167e-02 2.188114e-02 1.048500e-02 1.503315e+01
1.9 1.300314e-02 1.552049e-02 2.967183e-02 1.362027e-02 2.076491e+01
2.0 2.001076e-02 2.287483e-02 4.410523e-02 2.087507e-02 3.134798e+01


In [ ]:
#print "z =",zc,"lnH error =", np.sqrt(linalg.inv(Fishermat)[0,0]),\
    #"lnDA error =", np.sqrt(linalg.inv(Fishermat)[1,1]),\
    #"fsig8 error =",np.sqrt(linalg.inv(Fishermat)[2,2]),\
    #"bsig8 error =",np.sqrt(linalg.inv(Fishermat)[3,3]),\
    #"Ps error =",np.sqrt(linalg.inv(Fishermat)[4,4])

In [ ]:
#errorlist = [zc, np.sqrt(linalg.inv(Fishermat)[1,1])]
    
#    for eachitem in errorlist:
#        fo.write(str(eachitem))#+'\n')